## Kubeflow pipelines

This notebook goes through the steps of using Kubeflow pipelines using the Python3 interpreter (command-line) to preprocess, train, tune and deploy the babyweight model.


### 1. Start Hosted Pipelines and Notebook

To try out this notebook, first launch Kubeflow Hosted Pipelines and an AI Platform Notebooks instance.
Follow the instructions in this [README.md](pipelines/README.md) file.

### 2. Install necessary packages

In [1]:
%pip install --quiet kfp python-dateutil --upgrade

ERROR: tensorflow-probability 0.9.0 has requirement cloudpickle>=1.2.2, but you'll have cloudpickle 1.1.1 which is incompatible.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


Make sure to *restart the kernel* to pick up new packages (look for button in the ribbon of icons above this notebook)

## 3. [Optional] Build Docker containers

I have made my containers public, so you can simply use those.

In [ ]:
%%bash
cd pipelines/containers
#bash build_all.sh

### 4. Connect to the Hosted Pipelines

Visit https://console.cloud.google.com/ai-platform/pipelines/clusters
and get the hostname for your cluster.  You can get it by clicking on the Settings icon.
Alternately, click on the Open Pipelines Dashboard link and look at the URL.
Change the settings in the following cell

In [1]:
# CHANGE THESE
PIPELINES_HOST='447cdd24f70c9541-dot-us-central1.notebooks.googleusercontent.com'
PROJECT='ai-analytics-solutions'
BUCKET='ai-analytics-solutions-kfpdemo'

In [2]:
import kfp
client = kfp.Client(host=PIPELINES_HOST)
#client.list_pipelines()

### 5. Upload and execute pipeline

Upload to the Kubeflow pipeline cluster

In [7]:
from pipelines import mlp_babyweight

pipeline = client.create_run_from_pipeline_func(mlp_babyweight.train_and_deploy, 
                                                arguments={'project': PROJECT, 'bucket': BUCKET})

In [8]:
!gsutil cp pipelines/*.py gs://ai-analytics-solutions-kfpdemo/babyweight/preproc/tmp/staging/preprocess-babyweight-features-200202-000230.1580601751.591162

Copying file://pipelines/mlp_babyweight.py [Content-Type=text/x-python]...
/ [1 files][  4.3 KiB/  4.3 KiB]                                                
Operation completed over 1 objects/4.3 KiB.                                      


In [12]:
# Copyright 2020 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.